In [1]:
import numpy as np
%matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sp
from sklearn.datasets import make_moons
from sklearn.neighbors import kneighbors_graph
from scipy.sparse.csgraph import laplacian
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
sns.set_style("dark")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
from scipy.linalg import eigh

Using matplotlib backend: Qt4Agg


In [30]:
def energy(H,A,A_c):
    tmp = A-A_c
    lap = laplacian(A,normed=False)
    return np.trace(tmp.dot(tmp.T)) + np.trace(H.T.dot(lap).dot(H)) #+ np.trace(A.dot(A.T))
def solve_H(A,P,B,r,cluster=2,optimal = False,ld=1):
    lap = laplacian(A,normed=False)
    if optimal ==True:
        _,embedding = eigh(lap, eigvals=(0, cluster - 1))
        return embedding
    return np.linalg.solve(ld*(lap.T+lap)+r*np.identity(lap.shape[0]),r*(P-B))
def solve_A(H,A_c,ld=1):
    diag_grad = (H**2).sum(axis=1).reshape(-1,1)
    diag_grad = np.tile(diag_grad,(1,A_c.shape[1]))
    res = ld*H.dot(H.T) + ld * A_c  - diag_grad
    return res/4.0

def bregman_fit(data_x,n_neighbors=5,n_cluster=2,iteration=100,r=0.1,optimal=False,ld=1):
    adj_matrix = kneighbors_graph(data_x,n_neighbors=n_neighbors ,include_self=False).toarray()
    adj_matrix = 0.5*(adj_matrix+adj_matrix.T)
    laplacianMatrix =laplacian(adj_matrix,normed=False)
    A_c = adj_matrix.copy()
    #Initializaiton

    # H=np.random.randn(X.shape[0],self.n_cluster)
    H=np.arange(data_x.shape[0]*n_cluster).reshape(data_x.shape[0],n_cluster)
    P = H
    row,_ = H.shape
    B = np.zeros(shape=H.shape)
    energies  = np.zeros((3,iteration),dtype = np.float)
    changed = np.zeros((2,iteration),dtype=np.float)
    for i in range(iteration):
        currentEnergy = energy(H, adj_matrix, A_c)
        energies[0,i] = currentEnergy
        H_prev = H.copy()
        H = solve_H(adj_matrix,P,B,r,cluster=n_cluster,optimal=optimal,ld=ld)#np.linalg.solve((laplacianMatrix+laplacianMatrix.T)+r*np.identity(n=row),r*(P-B))

        new_adj_matrix = solve_A(H,A_c,ld=1)
        new_adj_matrix = 0.5*(new_adj_matrix+new_adj_matrix)
        
        changed[0,i] =np.sum(new_adj_matrix==adj_matrix)
        val = np.linalg.norm(new_adj_matrix-adj_matrix)

        changed[1,i] = val
        
#         interested_indices = np.where(adj_matrix>0)
#         adj_matrix[interested_indices] = new_adj_matrix[interested_indices]
        y_k = H + B
        u, d, v = np.linalg.svd(y_k, full_matrices=True)
        P = u.dot(np.eye(N=u.shape[1], M=v.shape[0])).dot(v)
        err = np.linalg.norm(H-H_prev)
        B = B + H - P
#         if err<0.000005:
#             print 'threshold meets %f at iteration %d'%(err,i)
#             break
        adj_matrix = new_adj_matrix
    print 'H norm difference %f at iteration %d'%(np.linalg.norm(H.dot(H.T)-np.identity(H.shape[0])),i)
    
    kmeans = KMeans(n_cluster)
    pred = kmeans.fit_predict(normalize(H))
    emb  = normalize(H)
    return emb,pred,energies,changed

In [3]:
#Create dataset
NOISE = 0.1
N_SAMPLE = 100
data_x, data_y =make_moons(n_samples=N_SAMPLE)
data_x_noisy , data_y_noisy  =make_moons(n_samples=N_SAMPLE,noise=NOISE)

In [32]:
#No Noise

ITERATION = 100
NEIGHBOR = 5
ld=0.1
emb,pred,energies,changed = bregman_fit(data_x=data_x,n_neighbors=NEIGHBOR,n_cluster=2,iteration=ITERATION
                                        ,ld=ld,r=ld/1e3)
fig,ax = plt.subplots(nrows=6,ncols=1)
fig.set_size_inches(14.5, 12.5)
ax[3].set_title('energy')
ax[3].plot(range(ITERATION),energies[0,:]/energies[0,:].max())
ax[0].set_title('Prediction')
ax[0].scatter(data_x[:,0],data_x[:,1],c=pred)
ax[1].set_title('embedding_predicted labeling')
ax[1].scatter(emb[:,0],emb[:,1],c=pred)
ax[2].set_title('embedding_true labeling')
ax[2].scatter(emb[:, 0], emb[:, 1], c=data_y)
ax[4].plot(range(ITERATION),changed[0,:],label = 'changed')
ax[5].plot(range(ITERATION),changed[1,:],label = 'norm')
ax[4].legend()
ax[5].legend()
print np.mean(pred!=data_y),
plt.show()

H norm difference 9.999890 at iteration 99
0.46


In [26]:
# Noise

ITERATION = 1000
NEIGHBOR = 10
R = 1e-8
dev =0
ld_noise = 1
base=1e5
# R=ld_noise/base

emb,pred,energies,changed = bregman_fit(data_x=data_x_noisy,n_neighbors=NEIGHBOR,n_cluster=2,iteration=ITERATION,
                                        r=R)
fig,ax = plt.subplots(nrows=6,ncols=1)
fig.set_size_inches(14.5, 12.5)
ax[3].set_title('energy')
ax[3].plot(range(ITERATION),energies[0,:]/energies[0,:].max())
ax[0].set_title('Prediction')
ax[0].scatter(data_x_noisy[:,0],data_x_noisy[:,1],c=pred)
ax[1].set_title('embedding_predicted labeling')
ax[1].scatter(emb[:,0],emb[:,1],c=pred)
ax[1].set_ylim([emb[:,1].min()-dev,emb[:,1].max()+dev])
ax[1].set_xlim([emb[:,0].min()-dev,emb[:,0].max()+dev])

ax[2].set_title('embedding_true labeling')
ax[2].scatter(emb[:, 0], emb[:, 1], c=data_y)
ax[2].set_ylim([emb[:,1].min()-dev,emb[:,1].max()+dev])
ax[2].set_xlim([emb[:,0].min()-dev,emb[:,0].max()+dev])
ax[4].plot(range(ITERATION),changed[0,:],label = 'changed')
ax[5].plot(range(ITERATION),changed[1,:],label = 'norm')
ax[4].legend()
ax[5].legend()
print np.mean(pred!=data_y_noisy)
plt.show()

threshold meets 0.000002 at iteration 266
H norm difference 9.949874 at iteration 266
0.89


In [31]:
# Noise with eigen

ITERATION = 1000
NEIGHBOR = 5
R = 0.00000001
dev =0

emb,pred,energies,changed = bregman_fit(data_x=data_x,n_neighbors=NEIGHBOR,n_cluster=2,
                                        optimal=True,iteration=ITERATION,r=R)
fig,ax = plt.subplots(nrows=6,ncols=1)
fig.set_size_inches(14.5, 12.5)
ax[3].set_title('energy')
ax[3].plot(range(ITERATION),energies[0,:]/energies[0,:].max())
ax[0].set_title('Prediction')
ax[0].scatter(data_x[:,0],data_x[:,1],c=pred)
ax[1].set_title('embedding_predicted labeling')
ax[1].scatter(emb[:,0],emb[:,1],c=pred)
ax[1].set_ylim([emb[:,1].min()-dev,emb[:,1].max()+dev])
ax[1].set_xlim([emb[:,0].min()-dev,emb[:,0].max()+dev])

ax[2].set_title('embedding_true labeling')
ax[2].scatter(emb[:, 0], emb[:, 1], c=data_y)
ax[2].set_ylim([emb[:,1].min()-dev,emb[:,1].max()+dev])
ax[2].set_xlim([emb[:,0].min()-dev,emb[:,0].max()+dev])
ax[4].plot(range(ITERATION),changed[0,:],label = 'changed')
ax[5].plot(range(ITERATION),changed[1,:],label = 'norm')
ax[4].legend()
ax[5].legend()
print np.mean(pred!=data_y)
plt.show()

H norm difference 9.899495 at iteration 999
0.64
